In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm
import scipy.stats as st
import seaborn as sns
import statistics  as sts
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from yellowbrick.model_selection import FeatureImportances
import chardet
from datetime import datetime
from datetime import date
import os
import random

In [56]:
df_itens = pd.read_csv('DNC_order_items_dataset.csv',index_col = 0)
df_reviews = pd.read_csv('DNC_order_reviews_dataset.csv',index_col = 0)
df_orders = pd.read_csv('DNC_orders_dataset.csv',index_col = 0)
df_produtos = pd.read_csv('DNC_products_dataset.csv',index_col = 0)

In [57]:
''''DNC_orders_dataset
order_id – Id do pedido de compra;
customer_id – Id do consumidor que realizou a compra;
order_status – Status da compra;

DNC_order_items_dataset
order_id - Id do pedido de compra;
order_item_id – Número de produtos em uma mesma compra;
product_id – Id do produto;
price - preço;

DNC_products_dataset
product_id – Id do produto;
product_category_name – Categoria do produto;
product_name_length – Tamanho do nome do produto;
product_description_length – comprimento do produto;
products_weight_g – peso do produto em g;
products_lenght_cm – comprimento do produto em cm;
products_height_cm – Altura do produto em cm;
product_widht_cm – largura do produto em cm;

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-57-97e42a1b0d24>, line 20)

In [58]:
df_itens.head(100)
df_itens.shape

(112650, 4)

In [59]:
df_itens['order_id'].value_counts()

8272b63d03f5f79c56e9e4120aec44ef    21
ab14fdcfbe524636d65ee38360e22ce8    20
1b15974a0141d54e36626dca3fdc731a    20
9ef13efd6949e4573a18964dd1bbe7f5    15
428a2f660dc84138d969ccd69a0ab6d5    15
                                    ..
98c5b3461f699d417f26d5dcfc210e1e     1
035a721e5bfc1bc3f4e1ba0d7923cc84     1
13c43b9af95067fe5366ed2f57afcf77     1
1e96a45243abd1f2da33000b50e61c50     1
1e721ce42c736da389dc1e266e969acd     1
Name: order_id, Length: 98666, dtype: int64

In [60]:
df_reviews.head()
df_reviews.shape

(99224, 3)

In [61]:
df_orders.head()

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [62]:
df_produtos.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,625.0,20.0,17.0,13.0


##Verificando informações

In [63]:
df_itens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       112650 non-null  object 
 1   order_item_id  112650 non-null  int64  
 2   product_id     112650 non-null  object 
 3   price          112650 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.3+ MB


In [64]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_id     99224 non-null  object
 1   order_id      99224 non-null  object
 2   review_score  99224 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.0+ MB


In [65]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      99441 non-null  object
 1   customer_id   99441 non-null  object
 2   order_status  99441 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


In [66]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32951 entries, 0 to 32950
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_weight_g            32949 non-null  float64
 5   product_length_cm           32949 non-null  float64
 6   product_height_cm           32949 non-null  float64
 7   product_width_cm            32949 non-null  float64
dtypes: float64(6), object(2)
memory usage: 2.3+ MB


In [102]:
df1=df_reviews.merge(df_itens, on=['order_id'], how='right')

In [105]:
df1 = df1.groupby(['review_id','order_id', 'review_score', 'product_id', 'price'], as_index=False ).first()
df1.head(50)

,review_id,order_id,review_score,product_id,price,order_item_id
0,0001239bc1de2e33cb583967c2ca4c67,fc046d7776171871436844218f817d7d,5.0,461f43be3bdf8844e65b62d9ac2c7a5a,145.00,1
1,0001cc6860aeaf5b9017fe4131a52e62,d4665434b01caa9dc3e3e78b3eb3593e,5.0,6e7f4ae007302e93c5610894712289bb,60.00,1
2,00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5.0,a39cc58c1b5926b6f9f378daa89f1315,150.00,1
3,00032b0141443497c898b3093690af51,04fb47576993a3cb0c12d4b25eab6e4e,5.0,af51d485dc5255ba2e18b21b550156e6,259.90,1
4,00034d88989f9a4c393bdcaec301537f,5f358d797a49fe2f24352f73426215f6,5.0,63d6e7ab30f482382c9dfbbccae7da54,68.89,1
5,000359bceee1b6fe876b30d35b5f9ef2,97868e4c884bc85f4501ae0a66181cc9,5.0,d1c427060a0f73f6b889a5c7c61f2ac4,149.00,1
6,00041b1f51e77a663df6db3f539d694f,998b1beee068744ac833dd8b31944623,1.0,c18dfe3165eaa55de9ec8f462cd7556f,55.00,1
7,00046a69550325aea5fb89f65c7387f2,9fbda7367628952bc36c3512c46d887b,5.0,b5b2072f757c9317caa9c32c3bbf393d,9.99,1
8,0005534973388c830bb858cfba83b17b,a589caa6892ceacc6bbf2f8cc30a8ad4,5.0,da93657c402ea46729003c282352d727,109.90,1
9,00055e36e9608fe969231e551983a69c,f5fea26ab547eec920e6f8ecdc5c37e4,5.0,f02eae5fd95b59b1ec82397e1ef7ccf1,169.90,1


In [106]:
count_top10 = df1.groupby(by='product_id', as_index=False) \
  .agg({'review_id': pd.Series.nunique}) \
  .rename(columns={'review_id': 'Review_count'})

In [107]:
count_top10.head()

,product_id,Review_count
0,00066f42aeeb9f3007548bb9d3f33c38,1
1,00088930e925c41fd95ebfe695fd2655,1
2,0009406fd7479715e4bef61dd91f2462,1
3,000b8f95fcb9e0096488278317764d19,2
4,000d9be29b5207b54e86aa1b1ac54872,1


## Recomendações simples

### Recomendação de itens mais/melhores avaliados

In [126]:
ratings_explicit = df1[df1['review_score'] != 0]

ratings_sum = pd.DataFrame(ratings_explicit.groupby(['product_id'])['review_score'].sum())
top1500_ratings = ratings_sum.sort_values('review_score', ascending = False).head(1500)


In [127]:
top1500_ratings.head()

,review_score
product_id,
99a4788cb24856965c36a24e339b6058,1800.0
aca2eb7d00ea1a7b8ebd4e68314663af,1759.0
422879e10f46682990de24d770e7f83d,1456.0
d1c427060a0f73f6b889a5c7c61f2ac4,1355.0
389d119b48cf3043d311335e499d9c6b,1295.0


In [128]:
top1500_ratings.merge(df_produtos, left_index = True,right_on = 'product_id')

,review_score,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
9661,1800.0,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,54.0,245.0,1383.0,50.0,10.0,40.0
13430,1759.0,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,44.0,903.0,2600.0,50.0,10.0,30.0
14051,1456.0,422879e10f46682990de24d770e7f83d,ferramentas_jardim,56.0,348.0,1550.0,30.0,22.0,30.0
8290,1355.0,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,59.0,1893.0,6550.0,20.0,20.0,20.0
30293,1295.0,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,59.0,341.0,1750.0,37.0,22.0,40.0
...,...,...,...,...,...,...,...,...,...
24389,43.0,d5140e1d258c83a340419c6522c7da22,moveis_decoracao,51.0,439.0,1500.0,42.0,10.0,32.0
16063,43.0,f73e87b204f0a4bcfa8edda38d11b334,automotivo,58.0,425.0,150.0,16.0,6.0,11.0
28660,43.0,c0350d6ac413eda4641bf92ab687f1b5,fashion_bolsas_e_acessorios,29.0,132.0,50.0,17.0,4.0,13.0
23463,43.0,d986e4bd747ca20b348abcd013cd1303,esporte_lazer,59.0,938.0,1200.0,80.0,14.0,14.0


In [119]:
df1.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'Media Avaliaçoes'}) \
    .set_index('product_id') \
    .sort_values('Media Avaliaçoes', ascending = False)

,Media Avaliaçoes
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
c6ac3472e33913332e3f394ed2ebec24,5.0
c6cae2869a248cef27c00f3340ac90a6,5.0
600add16084eeeead5c56279ccef0d7e,5.0
600dd8d6e08f57945bf0798abe1b1b5b,5.0
...,...
2fe1e09df32992e9e5041a2671233e20,1.0
2fc854e3ed45da2c8e6031bf79521203,1.0
b44cc626aea8d2a65572672cb050c083,1.0


### Recomendação de itens mais populares

In [110]:
df2=df1.merge(df_orders, on=['order_id'], how='left')

In [111]:
df2.head(50)

,review_id,order_id,review_score,product_id,price,order_item_id,customer_id,order_status
0,0001239bc1de2e33cb583967c2ca4c67,fc046d7776171871436844218f817d7d,5.0,461f43be3bdf8844e65b62d9ac2c7a5a,145.00,1,1458a88f3713f047a6d4a23a8ee0f3e5,delivered
1,0001cc6860aeaf5b9017fe4131a52e62,d4665434b01caa9dc3e3e78b3eb3593e,5.0,6e7f4ae007302e93c5610894712289bb,60.00,1,430629c51214778912efab67e023a8f1,delivered
2,00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5.0,a39cc58c1b5926b6f9f378daa89f1315,150.00,1,b1ff51be5960d3ed39bb63679f068555,delivered
3,00032b0141443497c898b3093690af51,04fb47576993a3cb0c12d4b25eab6e4e,5.0,af51d485dc5255ba2e18b21b550156e6,259.90,1,c158fb97a12041c9305718081a78b064,delivered
4,00034d88989f9a4c393bdcaec301537f,5f358d797a49fe2f24352f73426215f6,5.0,63d6e7ab30f482382c9dfbbccae7da54,68.89,1,e44e2e5c3ad690e075b368557195abb7,delivered
5,000359bceee1b6fe876b30d35b5f9ef2,97868e4c884bc85f4501ae0a66181cc9,5.0,d1c427060a0f73f6b889a5c7c61f2ac4,149.00,1,eaa701838126e86dd60ee4e44d274f21,delivered
6,00041b1f51e77a663df6db3f539d694f,998b1beee068744ac833dd8b31944623,1.0,c18dfe3165eaa55de9ec8f462cd7556f,55.00,1,4388eb52412b71e3b55ca093b0fca09b,delivered
7,00046a69550325aea5fb89f65c7387f2,9fbda7367628952bc36c3512c46d887b,5.0,b5b2072f757c9317caa9c32c3bbf393d,9.99,1,c4e670e1b3eddf57cd698f323332ba6c,delivered
8,0005534973388c830bb858cfba83b17b,a589caa6892ceacc6bbf2f8cc30a8ad4,5.0,da93657c402ea46729003c282352d727,109.90,1,a77486452cbf830a5f60aa0a1362e6af,delivered
9,00055e36e9608fe969231e551983a69c,f5fea26ab547eec920e6f8ecdc5c37e4,5.0,f02eae5fd95b59b1ec82397e1ef7ccf1,169.90,1,9d1f4537b5bf271b2acdfb484507104c,delivered


In [129]:
count = df2.groupby(by='product_id', as_index=False) \
  .agg({'order_id': pd.Series.nunique}) \
  .rename(columns={'order_id': 'Compras Realizadas'})

buy_count = count.set_index('product_id')
top1500_pop = buy_count.sort_values('Compras Realizadas', ascending = False).head(1500)

top1500_pop.merge(df_produtos, left_index = True,right_on = 'product_id')

,Compras Realizadas,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
9661,458,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,54.0,245.0,1383.0,50.0,10.0,40.0
13430,429,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,44.0,903.0,2600.0,50.0,10.0,30.0
14051,351,422879e10f46682990de24d770e7f83d,ferramentas_jardim,56.0,348.0,1550.0,30.0,22.0,30.0
8290,320,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,59.0,1893.0,6550.0,20.0,20.0,20.0
30293,310,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,59.0,341.0,1750.0,37.0,22.0,40.0
...,...,...,...,...,...,...,...,...,...
4192,10,b7a89537c40b1f5a18b5ba75dbb91cbe,instrumentos_musicais,49.0,1051.0,594.0,20.0,15.0,16.0
8078,10,c56400aa5823575d5a92a8f98077490a,moveis_escritorio,48.0,1043.0,9150.0,35.0,52.0,35.0
16679,10,c448475cc4766baebabe30ee6d5f3fb7,papelaria,50.0,105.0,450.0,20.0,30.0,20.0
31206,10,60567eaf5cdf794639e94508ec387a82,fashion_bolsas_e_acessorios,55.0,903.0,3350.0,62.0,20.0,40.0


In [130]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102230 entries, 0 to 102229
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   review_id      102230 non-null  object 
 1   order_id       102230 non-null  object 
 2   review_score   102230 non-null  float64
 3   product_id     102230 non-null  object 
 4   price          102230 non-null  float64
 5   order_item_id  102230 non-null  int64  
 6   customer_id    102230 non-null  object 
 7   order_status   102230 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 7.0+ MB


In [ ]:
# Criando e visualizando a matriz de relacionamento entre produtos

In [133]:
matriz_de_relacionamento_top_products_ratings = df2[df2["product_id"].isin(top1500_ratings.index.to_list())] \
  .pivot_table(index="product_id", columns="customer_id", values="review_score").fillna(0)

matriz_de_relacionamento_top_products_ratings.shape

(1500, 40480)

In [136]:
matriz_de_relacionamento_top_products_ratings.head(200)

customer_id,00012a2ce6f8dcda20d059ce98491703,000379cdec625522490c315e70c7a9fb,0004164d20a9e969af783496f3408652,00046a560d407e99b969756e0b10f282,00066ccbe787a588c52bd5ff404590e3,000e943451fc2788ca6ac98a682f2f49,000fd45d6fedae68fc6676036610f879,001028b78fd413e19704b3867c369d3a,001226b2341ef620415ce7bbafcfac28,0012a5c13793cf51e253f096a7e740dd,...,ffed827348cd4b083956ce2b634b763b,fff0f9fbcdf9cfe4d9328bd3a7af71d7,fff212062d600f2e1d53f3c5d4a25138,fff4f0c85a4d710abe06599e1ed08a42,fff5169e583fd07fac9fec88962f189d,fff5dd22d522cf28a902185817642a2e,fff906ecb75de5809be384e0f8d65e45,fffb97495f78be80e2759335275df2aa,fffecc9f79fd8c764f843e9951b11341,ffff42319e9b2d713724ae527742af25
product_id,,,,,,,,,,,,,,,,,,,,,
001b72dfd63e9833e8c02742adf472e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005030ef108f58b46b78116f754d8d38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
008cff0e5792219fae03e570f980b330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ba6d766f0b1d7b78a5ce3e1e033263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00baba5b58e274d0332a0c8a0a66f877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20f160a9cf2e856ee5eba259e9ae33f9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210a8305d79b2f5719cbafb2c026797f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211293b50935ced8d4370f552c482d74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Matriz de relacionamento e Modelos

## Recomendação Produtos similares - cliente estiver vendo um produto, e assim irá aparecer para o mesmo outros produtos similares.

In [139]:
from sklearn.metrics.pairwise import cosine_similarity


cos_products = cosine_similarity(matriz_de_relacionamento_top_products_ratings)

print(cos_products)
cos_products.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


(1500, 1500)

In [163]:
def recomendar(product_id):
  print(f"Produto de interesse: {product_id}")
  try:
    product_idx = matriz_de_relacionamento_top_products_ratings.index.tolist().index(product_id)
    closest_10_products = np.argsort(-cos_products[product_idx])[1:11]
    print(f"Retornando 10 produtos mais próximos de: {product_id}.")
    return matriz_de_relacionamento_top_products_ratings.index[closest_10_products].tolist()
  except ValueError as e:
    print(f"{product_id}, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados")
    return top1500_ratings.head(10).index.to_list()

In [164]:
recomendar('99a4788cb24856965c36a24e339b6058')

Produto de interesse: 99a4788cb24856965c36a24e339b6058
Retornando 10 produtos mais próximos de: 99a4788cb24856965c36a24e339b6058.


['35afc973633aaeb6b877ff57b2793310',
 'f2e53dd1670f3c376518263b3f71424d',
 '3e5201fe0d1ba474d9b90152c83c706c',
 '42a2bd596fda1baef5719cb74f73030c',
 '98b22c1129886cbd7fcae69d1c93e442',
 'eebbed5ed3b134eceb717496c47652ba',
 '9ad75bd7267e5c724cb42c71ac56ca72',
 'ab1f9387c0627dd24000bfbc54fdeee0',
 'acc444eb5ad26f79d5a11baa6a03c439',
 'aca2eb7d00ea1a7b8ebd4e68314663af']

In [ ]:
# Modelo criado para quando o cliente estiver vendo um produto, e assim irá aparecer para o mesmo outros produtos similares.

# Recomendação por Categoria - cliente estiver procurando um produto

In [168]:
buy_count = count.set_index('product_id')
top_pop = buy_count.sort_values('Compras Realizadas', ascending = False)

pop_cat = top_pop.merge(df_produtos, left_index = True,right_on = 'product_id')

In [169]:
pop_cat[pop_cat['product_category_name'] == 'pc_gamer']

,Compras Realizadas,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1628,5,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59.0,621.0,2839.0,19.0,16.0,18.0
7478,2,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60.0,1532.0,650.0,16.0,22.0,20.0
16930,1,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60.0,840.0,800.0,18.0,22.0,22.0


In [167]:
df3=df2.merge(df_produtos, on=['product_id'], how='left')
df3.head()

,review_id,order_id,review_score,product_id,price,order_item_id,customer_id,order_status,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0001239bc1de2e33cb583967c2ca4c67,fc046d7776171871436844218f817d7d,5.0,461f43be3bdf8844e65b62d9ac2c7a5a,145.00,1,1458a88f3713f047a6d4a23a8ee0f3e5,delivered,relogios_presentes,35.0,591.0,444.0,20.0,14.0,16.0
1,0001cc6860aeaf5b9017fe4131a52e62,d4665434b01caa9dc3e3e78b3eb3593e,5.0,6e7f4ae007302e93c5610894712289bb,60.00,1,430629c51214778912efab67e023a8f1,delivered,fraldas_higiene,49.0,154.0,1250.0,20.0,22.0,16.0
2,00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5.0,a39cc58c1b5926b6f9f378daa89f1315,150.00,1,b1ff51be5960d3ed39bb63679f068555,delivered,casa_construcao,53.0,646.0,3800.0,30.0,35.0,30.0
3,00032b0141443497c898b3093690af51,04fb47576993a3cb0c12d4b25eab6e4e,5.0,af51d485dc5255ba2e18b21b550156e6,259.90,1,c158fb97a12041c9305718081a78b064,delivered,malas_acessorios,57.0,674.0,22350.0,32.0,74.0,53.0
4,00034d88989f9a4c393bdcaec301537f,5f358d797a49fe2f24352f73426215f6,5.0,63d6e7ab30f482382c9dfbbccae7da54,68.89,1,e44e2e5c3ad690e075b368557195abb7,delivered,eletroportateis,56.0,1031.0,625.0,21.0,19.0,16.0


In [186]:
df3['product_category_name'].value_counts()

cama_mesa_banho                  10176
beleza_saude                      9011
esporte_lazer                     7862
informatica_acessorios            6901
moveis_decoracao                  6786
                                 ...  
cds_dvds_musicais                   12
la_cuisine                          12
pc_gamer                             8
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: product_category_name, Length: 73, dtype: int64

In [191]:
df3.shape

(102230, 15)

In [178]:
def recomendar_cat(product_id):
  print(f"Produto de interesse: {product_id}")
  try:
    categoria = pop_cat[pop_cat['product_id'] == product_id ]
    a = categoria['product_category_name'].to_list()
    
    df_aux = df3[df3["product_id"].isin(top1500_ratings.index.to_list())]
    matriz_de_relacionamento_top_products_pop = df_aux[df_aux['product_category_name'] == a[0] ] \
    .pivot_table(index="product_id", columns="customer_id", values="review_score").fillna(0)
    
    cos_products_cat = cosine_similarity(matriz_de_relacionamento_top_products_pop)
    
    product_idx = matriz_de_relacionamento_top_products_pop.index.tolist().index(product_id)
    closest_10_products_cat = np.argsort(-cos_products_cat[product_idx])[1:11]
    print(f"Retornando os produtos mais próximos de: {product_id}.")
    return matriz_de_relacionamento_top_products_pop.index[closest_10_products_cat].tolist()
  except ValueError as e:
    print(f"{product_id}, não está incluso na matriz de recomendação. Retornando os mais vendidos na categoria")
    return pop_cat[pop_cat['product_category_name'] == a[0]].head(10)['product_id'].to_list()


In [194]:
id_product_interesse_cat = "461f43be3bdf8844e65b62d9ac2c7a5a"
ids_10_mais_proximos_cat = recomendar_cat(id_product_interesse_cat)
ids_10_mais_proximos_cat

Produto de interesse: 461f43be3bdf8844e65b62d9ac2c7a5a
Retornando os produtos mais próximos de: 461f43be3bdf8844e65b62d9ac2c7a5a.


['1b474c650cb9407d32a1e066937b68fd',
 'bce474cc95b6e77657e10939b0b4248c',
 'ba74c6b75d2ad7503175809688d5a03c',
 'b84520a57891e7a8ae2c68741dcc7146',
 'b2e22c958b453a43a650a579dcac2e43',
 'b01d34f6d2545269d348181768eca049',
 'aa8d88eb4b9cb38894e33fa624c4287f',
 'a92930c327948861c015c919a0bcb4a8',
 'bfc0d01be79d9038c7720f983bf954e0',
 '02475368dfb38934fe55f574024fe1d7']

In [195]:
#Teste
categoria = pop_cat[pop_cat['product_id'] == '461f43be3bdf8844e65b62d9ac2c7a5a' ]
a = categoria['product_category_name'].to_list()
a[0]

'relogios_presentes'

In [190]:
#Teste
pop_cat[pop_cat['product_category_name'] == a[0]].head(10)
pop_cat['product_category_name'].shape

(32789,)